In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import year, to_timestamp
import xml.etree.ElementTree as ET

# Создание сессии Spark
spark = SparkSession.builder \
    .appName("yarl_lab_2") \
    .getOrCreate()

In [ ]:
tree = ET.parse("posts_sample.xml")
root = tree.getroot()
data = []

# Парсим xml файл
for row in root.findall('row'):
  # Получаем нужные поля
    date = row.attrib.get('CreationDate')
    tags = row.attrib.get('Tags')
    if tags is not None:
        # Т.к. в тэге поста может много языков - удаляем лишние символы и разбиваем на массив тэгов
        tags = tags.replace('<', '').replace('>', ' ').strip().split()
        for tag in tags:
            data.append((date, tag))

data

In [ ]:
# Создание схемы DataFrame
schema = StructType([
    StructField("CreationDate", StringType(), True),
    StructField("Tag", StringType(), True)
])

# Создание DataFrame
xml_df = spark.createDataFrame(data, schema=schema)

# Извлекаем год из доты
xml_df = xml_df.withColumn("Year", year(to_timestamp(xml_df["CreationDate"], "yyyy-MM-dd'T'HH:mm:ss.SSS")))

# Дропаем ненужный столбец
xml_df = xml_df.drop("CreationDate")

xml_df.show()


In [ ]:
schema = StructType([
    StructField("Tag", StringType(), True),
    StructField("wikipedia_url", StringType(), True)
])

# Читаем CSV файл в DataFrame
csv_df = spark.read.csv("programming-languages.csv", schema=schema)

# Добавляем индекс к DataFrame
csv_df = csv_df.rdd.zipWithIndex().toDF(["data", "index"])

# Извлекаем данные обратно в DataFrame
csv_df = csv_df.filter(csv_df.index != 0).select("data.*")

from pyspark.sql.functions import lower

csv_df = csv_df.withColumn("Tag", lower(csv_df["Tag"]))

csv_df.show()

In [ ]:
# Джоиним таблицы, чтобы убрать теги без языков программирования
df = xml_df.join(csv_df, on="Tag", how="inner")
df = df.drop("wikipedia_url")

df.show()

In [ ]:
from pyspark.sql import Window

# Группируем по году и языку, считаем количество упоминаний
language_counts = df.groupBy("Tag", "Year").count()

# Определяем окно для ранжирования
window_spec = Window.partitionBy("Year").orderBy(F.desc("count"))

# Добавляем ранг и фильтруем топ-10 языков для каждого года
top_languages = language_counts.withColumn("rank", F.row_number().over(window_spec)) \
    .filter(F.col("rank") <= 10) \
    .drop("rank")

top_languages = top_languages.orderBy("Year", F.desc("count"))

# Сохраняем результат в формате Parquet
top_languages.write.mode("overwrite").parquet("top_10_languages_by_year.parquet")
